In [1]:
import pandas as pd
import psycopg2
import glob
import json
import csv

In [2]:
JSON_FILES = "json/*.json"

In [3]:
row_data = []
for fp in glob.glob(JSON_FILES):
    try:
        with open(fp, 'r') as f:
            data = json.load(f)
        for track in data['items']:
            song_data = [
                track['track']['artists'][0]['name'],
                track['track']['artists'][0]['uri'],
                track['track']['name'],
                track['track']['uri'],
                int(track['track']['duration_ms']),
                track['track']['popularity'],
                track['played_at']
            ]
            row_data.append(song_data)
    except:
        pass

In [4]:
columns = ['artistName','artistUri','songName','songUri','songDuration','songPopularity','played_at']
df = pd.DataFrame(row_data, columns=columns)

In [5]:
df.drop_duplicates('played_at', inplace=True)
df = df.sort_values('played_at', ascending=False)
df.head(5)

,artistName,artistUri,songName,songUri,songDuration,songPopularity,played_at
397,Giom,spotify:artist:43Of1c5VKtUcPmNZTLu2it,Hot Rabbits - Spiritchaser Remix,spotify:track:22QGqIGkPgCySsWIfiLHaS,482000,5,2021-04-19T01:03:50.146Z
398,Moe Turk,spotify:artist:12rv3gD3SpWQRRkswqlwvF,Lost,spotify:track:2t7MoYPZdgmGtoTyskyVr2,337735,24,2021-04-18T23:57:22.876Z
399,Etta James,spotify:artist:0iOVhN3tnSvgDbcg25JoJb,A Sunday Kind Of Love,spotify:track:0zGLlXbHlrAyBN1x6sY0rb,196133,71,2021-04-18T23:56:39.622Z
400,Alina K.,spotify:artist:1Pg8rh0BcD7t64YOWGFiwX,Walking Your Path - Full Intention Sunset Mix,spotify:track:6VGLQIcZm8tn4tBlaKsbrd,320702,37,2021-04-18T23:53:24.720Z
401,Tuplain,spotify:artist:7rju9kEAn7gtTnjCMtHhMj,The Vibes,spotify:track:13o9PO4h9qKcWTjwye6UCo,289970,32,2021-04-18T15:53:41.654Z


In [6]:
df.to_csv('songplay.csv',index=False, sep = '|')

In [7]:
conn = psycopg2.connect("host=localhost dbname=spotify user=postgres password=###")
cur = conn.cursor()

In [8]:
cur.execute("DROP TABLE IF EXISTS songplay")

In [9]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS songplay(
        artistName VARCHAR NOT NULL,
        artistUri VARCHAR NOT NULL,
        songName VARCHAR NOT NULL,
        songUri VARCHAR NOT NULL,
        songDuration BIGINT NOT NULL,
        songPopularity INT NOT NULL,
        playedAt TIMESTAMP NOT NULL
)
""")

In [10]:
with open('songplay.csv', 'r') as f:
    next(f)
    cur.copy_from(f, 'songplay', sep='|', null='')
    conn.commit()

In [11]:
cur.execute("SELECT * FROM songplay LIMIT 1")
cur.fetchall()

[('Giom',
  'spotify:artist:43Of1c5VKtUcPmNZTLu2it',
  'Hot Rabbits - Spiritchaser Remix',
  'spotify:track:22QGqIGkPgCySsWIfiLHaS',
  482000,
  5,
  datetime.datetime(2021, 4, 19, 1, 3, 50, 146000))]

In [12]:
cur.close()